Find the time and value of max load for each of the regions COAST, EAST, FAR_WEST, NORTH, NORTH_C, SOUTHERN, SOUTH_C, WEST
and write the result out in a csv file, using pipe character as the delimiter.

An example output can be seen in the "example.csv" file

In [18]:
import xlrd
import os
import csv
from zipfile import ZipFile

datafile = "2013_ERCOT_Hourly_Load_Data.xls"
outfile = "2013_Max_Loads.csv"

# open given file + extract
def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile), 'r') as myzip:
        myzip.extractall()

#def parse_file(datafile):
    # open the workbook w/ xlrd library's open_workbook method and get the 1st sheet
workbook = xlrd.open_workbook(datafile)
sheet = workbook.sheet_by_index(0)

headers = ['Station','Year','Month','Day','Hour','Max Load']
stations = []
loads = []
    # YOUR CODE HERE
    # Remember that you can use xlrd.xldate_as_tuple(sometime, 0) to convert
    # Excel date to Python tuple of (year, month, day, hour, minute, second)
for row in range(sheet.nrows):
    for col in range(sheet.ncols):
        if row == 0:
            if col > 0 and col < sheet.ncols - 1:
                loads.append(sheet.cell_value(row,col))
                #+'|')
        if row > 0:
            if col > 0 and col < sheet.ncols - 1:
                if sheet.cell_value(row,col) > sheet.cell_value((row-1),(col-1):
                    loads.append(sheet.cell_value(row,col))
                    
stations

SyntaxError: invalid syntax (<ipython-input-18-5b920ef2289a>, line 33)

In [13]:
loads

[]

In [14]:
#data.append(loads)
#len(data)
range(0,len(data))

range(0, 8)

In [15]:
with open(outfile, 'wt') as csvfile:
    data_writer = csv.writer(csvfile, delimiter ='|',quotechar =',',quoting=csv.QUOTE_MINIMAL)
    
    for i in data:
        #print(i)
        data_writer.writerow(headers)
        data_writer.writerow(stations)
        data_writer.writerow(loads)

In [55]:
        for col in range(sheet.ncols):
            if row > 0:
                if col == 1:
                    coast.append(sheet.cell_value(row,col))
                elif col == 0:
                    times.append(sheet.cell_value(row,col))    
    return data

def save_file(data, filename):
    # YOUR CODE HERE

    
def test():
    open_zip(datafile)
    data = parse_file(datafile)
    save_file(data, outfile)

    number_of_rows = 0
    stations = []

    ans = {'FAR_WEST': {'Max Load': '2281.2722140000024',
                        'Year': '2013',
                        'Month': '6',
                        'Day': '26',
                        'Hour': '17'}}
    correct_stations = ['COAST', 'EAST', 'FAR_WEST', 'NORTH',
                        'NORTH_C', 'SOUTHERN', 'SOUTH_C', 'WEST']
    fields = ['Year', 'Month', 'Day', 'Hour', 'Max Load']

    with open(outfile) as of:
        csvfile = csv.DictReader(of, delimiter="|")
        for line in csvfile:
            station = line['Station']
            if station == 'FAR_WEST':
                for field in fields:
                    # Check if 'Max Load' is within .1 of answer
                    if field == 'Max Load':
                        max_answer = round(float(ans[station][field]), 1)
                        max_line = round(float(line[field]), 1)
                        assert max_answer == max_line

                    # Otherwise check for equality
                    else:
                        assert ans[station][field] == line[field]

            number_of_rows += 1
            stations.append(station)

        # Output should be 8 lines not including header
        assert number_of_rows == 8

        # Check Station Names
        assert set(stations) == set(correct_stations)

        
if __name__ == "__main__":
    test()


IndentationError: expected an indented block (<ipython-input-55-f810de686573>, line 13)